In [6]:
import os
from pymilvus import MilvusClient
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain import hub
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus
from langchain import hub
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import requests
import sys
import io
import json
import shutil
import sys
import datetime
import subprocess
import sys
import os
import math
import base64
from time import gmtime, strftime
import random, string
import time
import psutil
import base64
import uuid
import socket
import os
from pymilvus import connections
from pymilvus import utility
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import torch
from torchvision import transforms
from PIL import Image
import timm
from sklearn.preprocessing import normalize
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from IPython.display import display
from pymilvus import MilvusClient
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
import json
import requests
from datetime import datetime, timedelta
from langchain_milvus import Milvus
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from dotenv import load_dotenv
load_dotenv(verbose=True)
from langchain_core.globals import set_verbose, set_debug

# Disable verbose logging
set_verbose(False)

# Disable debug logging
set_debug(False)

### Setup environment and constants

DIMENSION = 384 
MILVUS_URL = "http://192.168.1.153:19530" 

slack_token = os.environ.get("SLACK_BOT_TOKEN")
client = WebClient(token=slack_token)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
SC_COLLECTION_NAME = "nycstreetcameras"
TODAYS_DATE = str( datetime.today().strftime('%Y-%m-%d') ) 
YESTERDAYS_DATE = (datetime.now() - timedelta(1)).strftime('%Y-%m-%d')

### Environment Variables needed
os.environ["LANGCHAIN_HUB_API_URL"] = "https://api.hub.langchain.com"
os.environ["LANGCHAIN_HUB_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_HUB_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.hub.langchain.com"
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

### Turn off slack warnings
os.environ["SKIP_SLACK_SDK_WARNING"] = "false"

# https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.milvus.Milvus.html 

/Users/timothyspann/Downloads/code/milvusvenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
### Set up connection to Milvus for NYC Street Cameras, weather text vector
vector_store = Milvus(
    embedding_function=embeddings,
    collection_name=SC_COLLECTION_NAME,
    primary_field = "id",
    vector_field = "weather_text_vector",
    text_field="weatherdetails",
    connection_args={"uri": MILVUS_URL},
)

In [12]:
# https://zilliz.com/blog/building-multilingual-rag-milvus-langchain-openai
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

retriever = vector_store.as_retriever(collection =  SC_COLLECTION_NAME)

llm = Ollama(
    model="llama3",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    stop=["<|eot_id|>"],
)

template="""
   Use the context to build detailed weather reports for the location asked for.
   {location}
   Context: {context}
   Answer:"""
  
prompt = ChatPromptTemplate.from_template(template)

chain = (
      {"context": retriever, "location": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

response = chain.invoke("What is the current weather in detail from Central park.")


try:
    slackresponse = client.chat_postMessage(mrkdwn=True, channel="C06NE1FU6SE", text="", 
                                        blocks=[{"type": "section","text": {"type": "mrkdwn","text": str(response) +"\n" }}])

    print("\n\nSLACKREPONSE: \n" + str(slackresponse))
except SlackApiError as e:
    # You will get a SlackApiError if "ok" is False
    print("Slack failed")
        

**Current Weather Report for New York City, Central Park [KNYC]**

As of 06:51 am EDT on August 29th, the current weather observation at KNVC in NY is:

* **Weather Condition:** Light Rain
* **Temperature:** 74°F (23.3°C)
* **Dew Point:** 61°F (16.1°C)
* **Relative Humidity:** 64%
* **Wind Speed:** 6,999 ft/min (21.4 m/s or 78 km/h)
* **Visibility:** 10.00 miles (16.09 kilometers)
* **Elevation:** 7 feet (2.13 meters) above sea level
* **Altimeter Reading:** 1,021.4 millibars

**Additional Information:**

The weather is cloudy with scattered light rain showers and a gentle breeze. The relative humidity is relatively high at 64%, indicating a humid environment.

Please note that these readings are based on the provided data and might not reflect the actual weather conditions at the time of observation.{'ok': True, 'channel': 'C06NE1FU6SE', 'ts': '1725129936.055319', 'message': {'user': 'U0733NGPTNZ', 'type': 'message', 'ts': '1725129936.055319', 'bot_id': 'B073BDWNZU5', 'app_id': 'A06M3